In [1]:
import open3d as o3d
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transforms3d
import copy

In [2]:
static_npy = "/home/saby/Projects/ati/data/data/datasets/Real_World/pair_transform/static_out_npy/0.npy"
dynamic_npy = "/home/saby/Projects/ati/data/data/datasets/Real_World/pair_transform/dynamic_out_npy/0.npy"

print("starting")
static_arr = np.load(static_npy)
print("static done")
dynamic_arr = np.load(dynamic_npy)
print("dynamic done")

starting
static done
dynamic done


In [3]:
static_arr.shape

(14000, 16, 1024, 4)

In [4]:
dynamic_arr.shape

(14000, 16, 1024, 4)

In [13]:
static_img = static_arr[1]
dynamic_img = dynamic_arr[1]

In [21]:
static_pcd = o3d.geometry.PointCloud()
static_pcd.points = o3d.utility.Vector3dVector(static_img.reshape(-1,4)[:,:3])

dynamic_pcd = o3d.geometry.PointCloud()
dynamic_pcd.points = o3d.utility.Vector3dVector(dynamic_img.reshape(-1,4)[:,:3])

In [18]:
# Function to get transformation matrix for a given pose
def pose2matrix(translation_list, rotation_angle_list, zoom_list=[1,1,1]):
    # trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom_vec = np.array(zoom_list)
    # transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom_vec)
    transform_mat = transforms3d.affines.compose(translation_list, rot_mat, zoom_list)
    return transform_mat

def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()
    
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [16]:
draw_registration_result(static_pcd, dynamic_pcd, 0, 0, 0)

JVisualizer with 2 geometries

In [24]:
draw_pcd(dynamic_pcd.paint_uniform_color([0, 0, 1]))

JVisualizer with 1 geometries

In [25]:
draw_pcd(static_pcd.paint_uniform_color([1, 0, 0]))

JVisualizer with 1 geometries